In [1]:
from collections import deque
import heapq
import numpy as np
import random

On initialise le jeu avec une configuration aléatoire.

In [2]:
def initialisation():
    plateau_initial = []
    while len(plateau_initial) < 9:
        a = random.randint(0, 8)
        if a not in plateau_initial:
            plateau_initial.append(a)

    return plateau_initial

#jeu_initial=initialisation()
#print(jeu_initial)
jeu_initial=[8,9,7,5,0,1,3,2,4]

Maintenant, on peut créer notre liste de toutes les configurations accessibles à partir de l'initiale.

In [3]:
def position_0(grille):
    for i in range(9):
        if grille[i]== 0 :
            position = np.array([i//3 , i%3])
            indice = i
            break
    return position , indice

In [4]:
def configurations_atteignables(probleme:list):

    probleme_t = tuple(probleme)

    configurations_atteignables = set()  #on y mettra toutes les configurations qui sont accessibles depuis le problème initial
    configurations_atteignables.add(probleme_t)

    arretes = {} # dictionnaire avec configuration possible en clé et en valeurs la liste des configurations auxquelles elle est reliée par un mouvement élémentaire


    voisins = [probleme] #on y met les configuartions atteignables que l'on n'a pas encore visité
    heapq.heapify(voisins) # on initialise la pile
    mouvements_possibles = [np.array([1,0]) , np.array([-1,0]) , np.array([0,1]) , np.array([0,-1])] #mouvements élémentaires possibles

    while len(voisins)>0 :  
        config = heapq.heappop(voisins) 
        position , indice = position_0(config)
        config_t = tuple(config)  # on a besoin d'un type hashable
        arretes[config_t] = []
        for k in mouvements_possibles :
            nouvelle_position = position + k
            i , j = position[0] , position[1] 
            i_nouv , j_nouv = nouvelle_position[0], nouvelle_position[1] 
            indice_nouv = 3*i_nouv + j_nouv

            if (i_nouv <= 2 ) and (j_nouv <= 2 ) and (i_nouv >= 0 ) and (j_nouv >= 0) :

                nouv_config = config.copy()
                nouv_config[indice_nouv] , nouv_config[indice] = config[indice] , config[indice_nouv]
                nouv_config_t = tuple(nouv_config) # on a besoin d'un type hashable

                if nouv_config_t not in configurations_atteignables :
                    configurations_atteignables.add(nouv_config_t)
                    heapq.heappush(voisins, nouv_config)
                    arretes[config_t].append(nouv_config_t)
    
    return configurations_atteignables , arretes


In [5]:
configurations_atteignables_, arretes = configurations_atteignables(jeu_initial)

print(type(list(configurations_atteignables_)))

<class 'list'>


Si la solution est atteignable, on lance alors Dijkstra pour trouver un chemin reliant les configurations initiales et finales.

In [6]:
########### dijkstra_path ##############


def dijkstra_path(g : dict, v_init,v_fin):
  visited = {x : False for x in g}  #dico des sommets visités {config:Bool}
  pred = {x : None for x in g}      #dico des predecesseurs
  dist = {x : float('inf') for x in g} #dico des distances
  dist[v_init] = 0  #v_init est à distance 0 de lui-même
  hq = [(0, v_init)]
  heapq.heapify(hq)  # création de la FP
  N=0   # compteur des sommets visités
  while len(hq)>0 and not(visited[v_fin]): # visite des sommets
    dv,v=heapq.heappop(hq)  # extraction du sommet de prio min
    if not visited[v]:   # s'il n'est pas déjà visité,
      visited[v] = True   # on le visite NAO
      N+=1  #maj du compteur
      for w in g[v]:
# parcours des voisins non visités de v car g[v] est liste des sommets atteignables à partir de v (et l'arrête est de poids 1)
        if not visited[w]:
          dw = dv + 1    # relâchement de l'arête (v,w)
          if dw < dist[w]:
            dist[w] = dw   # maj de la distance min
            pred[w] = v    # maj du prédécesseur
            heapq.heappush(hq, (dw, w)) # ajout dans la FP
  if not visited[v_fin]: # cas où v_fin n'est pas accessible
    print("Pas de solution, en partant de "+str(v_fin))
  else:   #construction du chemin
    C=[v_fin]
    while C[0]!= v_init:
      w=pred[C[0]]
      C=[w]+C
    return N,dist[v_fin],C

Il faut tout de même vérifier que la solution ie [1,2,3,4,5,6,7,8,0] est dans les configurations atteignables.

In [9]:
solution = tuple([1,2,3,4,5,6,7,8,0])
if solution not in configurations_atteignables_ : 
    print(dijkstra_path(arretes, tuple(jeu_initial), solution))
    print("Il n'y a pas de solution à partir de cette configuration initiale...")
else :
    N, dist, Pb_résolu=dijkstra_path(arretes, tuple(jeu_initial), solution)
    n=len(Pb_résolu)
    print(f"la solution se fait en {n} coups")
    for i in range(n):
        print(i+1, Pb_résolu[i]) 

KeyError: (1, 2, 3, 4, 5, 6, 7, 8, 0)